In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import matplotlib.pyplot as plt # plotting
import matplotlib.image as mpimg # images
import numpy as np #numpy
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow.compat.v2 as tf #use tensorflow v2 as a main 
import tensorflow.keras as keras # required for high level applications
from sklearn.model_selection import train_test_split # split for validation sets
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.preprocessing import normalize # normalization of the matrix
import scipy
import pandas as pd
import unicodedata, re, string
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob
import itertools

In [ ]:
true = pd.read_csv('/kaggle/input/fake-and-real-news-dataset/True.csv')
fake = pd.read_csv('/kaggle/input/fake-and-real-news-dataset/Fake.csv')

true['label'] = 0
fake['label'] = 1

data = pd.concat([true, fake], axis=0)

In [ ]:
def show_history(history):
    plt.figure()
    for key in history.history.keys():
        plt.plot(history.epoch, history.history[key], label=key)
    plt.legend()
    plt.tight_layout()

In [ ]:
class Mish(keras.layers.Activation):
    def __init__(self, activation, **kwargs):
        super(Mish, self).__init__(activation, **kwargs)
        self.__name__ = 'Mish'
def mish(inputs):
    return inputs * tf.math.tanh(tf.math.softplus(inputs))
keras.utils.get_custom_objects().update({'mish': Mish(mish)})

In [ ]:
data.head()

In [ ]:
data = data.drop(['title', 'subject', 'date'], axis=1)

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
sns.countplot(x='label', data=data)

In [ ]:
data.label.value_counts()

In [ ]:
data.head()

In [ ]:
for i,x in enumerate(data['text']):
    if i > 5:
        break
    print(i)
    print(x)
    print('---------')

In [ ]:
def normalize(data):
    normalized = []
    for i in data:
        i = i.lower()
        # get rid of urls
        i = re.sub('https?://\S+|www\.\S+', '', i)
        # get rid of non words and extra spaces
        i = re.sub('\\W', ' ', i)
        i = re.sub('\n', '', i)
        i = re.sub(' +', ' ', i)
        i = re.sub('^ ', '', i)
        i = re.sub(' $', '', i)
        normalized.append(i)
    return normalized
data['new_text'] = normalize(data['text'])

In [ ]:
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)
    
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)

data['new_text']=data['new_text'].apply(remove_stopwords)

In [ ]:
data.head()

In [ ]:
data['length'] = data.text.apply(len)
sns.barplot(x='label', y='length', data = data)

In [ ]:
data['length'] = data.new_text.apply(len)
sns.barplot(x='label', y='length', data = data)

In [ ]:
for i,x in enumerate(data['new_text']):
    if i > 10:
        break
    print(i)
    print(x)
    print('---------')

In [ ]:
all_words = list(itertools.chain(*data.new_text))

In [ ]:
max(data.new_text.apply(len))

Fake

In [ ]:
from wordcloud import WordCloud
plt.figure(figsize = (20,20))
wc = WordCloud(max_words = 10000 , width = 1600 , height = 800).generate(" ".join(data[data.label == 1].new_text))
plt.imshow(wc , interpolation = 'bilinear')

True

In [ ]:
plt.figure(figsize = (20,20)) 
wc = WordCloud(max_words = 10000 , width = 1600 , height = 800).generate(" ".join(data[data.label == 0].new_text))
plt.imshow(wc , interpolation = 'bilinear')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data["new_text"], data["label"], test_size=0.20, random_state=51)

In [ ]:
from tensorflow import string as tf_string
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

embedding_dim = 128 # Dimension of embedded representation - this is already part of latent space, there is captured some dependecy among words, we are learning this vectors in ANN
vocab_size = 10000 # Number of unique tokens in vocabulary
sequence_length = 30 # Output dimension after vectorizing - words in vectorited representation are independent

vect_layer = TextVectorization(max_tokens=vocab_size, output_mode='int', output_sequence_length=sequence_length)
vect_layer.adapt(X_train.values)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=13)

In [ ]:
print('Vocabulary example: ', vect_layer.get_vocabulary()[:10])
print('Vocabulary shape: ', len(vect_layer.get_vocabulary()))

In [ ]:
from tensorflow.compat.v1.keras.layers import CuDNNGRU, CuDNNLSTM
from tensorflow.keras.layers import LSTM, GRU,SimpleRNN, Bidirectional

input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_v)
x = LSTM(64, activation='mish', return_sequences=True)(emb)
x = GRU(64, activation='mish', return_sequences=True)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, 'mish')(x)
x = keras.layers.Dense(64, 'mish')(x)
x = keras.layers.Dense(32, 'mish')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()

model.compile(optimizer='rmsprop', loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_v)
x = LSTM(64, activation='relu', return_sequences=True)(emb)
x = GRU(64, activation='relu', return_sequences=True)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, 'relu')(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model2 = keras.Model(input_layer, output_layer)
model2.summary()

model2.compile(optimizer='rmsprop', loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_v)
x = SimpleRNN(64, return_sequences=True)(emb)
x = SimpleRNN(32)(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.5)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model3 = keras.Model(input_layer, output_layer)
model3.summary()

model3.compile(optimizer='rmsprop', loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)
batch_size = 64
epochs = 7

In [ ]:
history = model.fit(X_train.values, y_train.values, validation_data=(X_valid.values, y_valid.values), callbacks=[es], epochs=epochs, batch_size=batch_size)

In [ ]:
history2 = model2.fit(X_train.values, y_train.values, validation_data=(X_valid.values, y_valid.values), callbacks=[es], epochs=epochs, batch_size=batch_size)

In [ ]:
history3 = model3.fit(X_train.values, y_train.values, validation_data=(X_valid.values, y_valid.values), callbacks=[es], epochs=epochs, batch_size=batch_size)

In [ ]:
show_history(history)

In [ ]:
show_history(history2)

In [ ]:
show_history(history3)

In [ ]:
y_test_loss, accuracy = model.evaluate(X_test, y_test)

In [ ]:
y_test_loss, accuracy = model2.evaluate(X_test, y_test)

In [ ]:
y_test_loss, accuracy = model3.evaluate(X_test, y_test)

In [ ]:
y_pred = model.predict(X_test).ravel()
y_pred2 = model2.predict(X_test).ravel()
y_pred3 = model3.predict(X_test).ravel()

In [ ]:
y_pred = [1 if x >= 0.5 else 0 for x in y_pred]
y_pred2 = [1 if x >= 0.5 else 0 for x in y_pred2]
y_pred3 = [1 if x >= 0.5 else 0 for x in y_pred3]

In [ ]:
model1_score = accuracy_score(y_true=y_test, y_pred=y_pred)
model2_score = accuracy_score(y_true=y_test, y_pred=y_pred2)
model3_score = accuracy_score(y_true=y_test, y_pred=y_pred3)

In [ ]:
print(classification_report(y_true=y_test, y_pred=y_pred))
print(confusion_matrix(y_true=y_test, y_pred=y_pred))
print(classification_report(y_true=y_test, y_pred=y_pred2))
print(confusion_matrix(y_true=y_test, y_pred=y_pred2))
print(classification_report(y_true=y_test, y_pred=y_pred3))
print(confusion_matrix(y_true=y_test, y_pred=y_pred3))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

#Creating the bag of words
bow_article = CountVectorizer().fit(data['new_text'])

article_vect = bow_article.transform(data['new_text'])
#TF-IDF

from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(article_vect)
news_tfidf = tfidf_transformer.transform(article_vect)
print(news_tfidf.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X = news_tfidf
y = data['label']
X_train_N, X_test_N, y_train_N,y_test_N= train_test_split(X, y, test_size=0.2)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
mnb.fit(X_train_N, y_train_N)
mnb_score = mnb.score(X_test_N,y_test_N)
print(mnb_score)
print(classification_report(y_true=y_test_N, y_pred=mnb.predict(X_test_N)))
print(confusion_matrix(y_true=y_test_N, y_pred=mnb.predict(X_test_N)))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc=DecisionTreeClassifier(criterion='entropy')
dtc.fit(X_train_N,y_train_N)
dtc.score(X_test_N,y_test_N)
dtc_score = mnb.score(X_test_N,y_test_N)
print(dtc_score)
print(classification_report(y_true=y_test_N, y_pred=dtc.predict(X_test_N)))
print(confusion_matrix(y_true=y_test_N, y_pred=dtc.predict(X_test_N)))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knc = KNeighborsClassifier(3)
knc.fit(X_train_N,y_train_N)
knc.score(X_test_N,y_test_N)
knc_score = mnb.score(X_test_N,y_test_N)
print(knc_score)
print(classification_report(y_true=y_test_N, y_pred=knc.predict(X_test_N)))
print(confusion_matrix(y_true=y_test_N, y_pred=knc.predict(X_test_N)))

In [ ]:
print("model1",model1_score)
print("model2",model2_score)
print("model3",model3_score)
print("MultinomialNB", mnb_score)
print("DecisionTreeClassifier", dtc_score)
print("KNeighborsClassifier", knc_score)